<a href="https://colab.research.google.com/github/victor555ops/Clases_de_Python/blob/main/Clasificador__de_numeros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
#Descargar set de datos de MNIST 
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

In [ ]:
#obtener en variables separadas los datos de entrenamiento
datos_entrenamiento, datos_pruebas = datos['train'], datos['test'] 

In [ ]:
def normalizar(imagenes, etiquetas):
  imagenes=tf.cast(imagenes, tf.float32)
  imagenes/=255 #aqui se pasa de 0-255 a 0-1
  return imagenes, etiquetas

In [ ]:
#Normalizar los datos de entrenamiento con la funcion 
datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_prueba = datos_pruebas.map(normalizar)

In [ ]:
#Agregar a cache (usar memoriaen lugar de disco, entrenamiento)
datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

clases = ['0','1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
from keras.api._v2.keras import layers
from types import UnionType
modelo=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(units=50, activation='relu'),
    tf.keras.layers.Dense(units=50, activation='relu'),
])

In [ ]:
#compilar modelo
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
#los numeros de datos de entrenamiento
num_datos_entrenamiento = metadatos.splits['train'].num_examples
num_datos_pruebas = metadatos.splits['test'].num_examples

In [ ]:
#trabajar por lotes
TAMANO_LOTE=32

In [ ]:
#shuffle y repeat hacen que los datos esten mezaclados de manera aleatoria 
#para que el entrenamiento no se aprenda las cosas en orden 
datos_entrenamiento=datos_entrenamiento.repeat().shuffle(num_datos_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

In [ ]:
#Realizar el entrenamiento

import math
historical = modelo.fit(
    datos_entrenamiento,
    epochs = 60,
    steps_per_epoch=math.ceil(num_datos_entrenamiento/TAMANO_LOTE)
)